### Building A Chatbot
In this project, We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.


Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for :
- Conversational RAG : Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take acions

This Project will cover the basic which will be helpful for those two more advance topics.

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_KEY")


In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000139E64049E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000139E6406FF0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Vicky and I am a Cheif AI Engineer")])

AIMessage(content="Hello Vicky! It's nice to meet you. \n\nThat's quite a title!  As a Chief AI Engineer, I imagine you're involved in some fascinating work.  What are some of the most exciting projects you're currently working on?  \n\nI'm always eager to learn more about the applications of AI. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 23, 'total_tokens': 98, 'completion_time': 0.136363636, 'prompt_time': 0.000212419, 'queue_time': 0.015262952, 'total_time': 0.136576055}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-02cc079b-296b-411c-8d38-81a865a02fd1-0', usage_metadata={'input_tokens': 23, 'output_tokens': 75, 'total_tokens': 98})

In [8]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Vicky and I am a Cheif AI Engineer"),
    AIMessage(content="Hello Vicky! It's nice to meet you. \n\nThat's quite a title!  As a Chief AI Engineer, I imagine you're involved in some fascinating work. What are some of the most exciting projects you're currently working on?  \n\nI'm always eager to learn more about the applications of AI. \n\n"),
    HumanMessage(content="Hey What is my name and what do I do?")
])


AIMessage(content="According to our conversation, your name is Vicky and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to know or discuss?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 115, 'total_tokens': 153, 'completion_time': 0.069090909, 'prompt_time': 0.00362316, 'queue_time': 0.011676189, 'total_time': 0.072714069}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a93643aa-57a2-4f98-ab65-c564cdcce0be-0', usage_metadata={'input_tokens': 115, 'output_tokens': 38, 'total_tokens': 153})

In [9]:
!pip install langchain_community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.4 MB 2.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 1.8 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.4 MB 1.8 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 3.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.1 MB 3.5 MB/s eta 0:00:01
   -------------------------------------

### Message History
We can use a message History class to wrap out model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastores. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [11]:
store={}

def get_seesion_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_seesion_history)

In [12]:
config = {"configurable":{"session_id":"chat1"}}


In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is vicky and I am a chief AI Engineer")],
    config=config
)

In [14]:
response

AIMessage(content="Hi Vicky, it's great to meet you! That's a fantastic title.  \n\nWhat kind of exciting AI projects are you working on these days?  I'm always eager to learn about the innovative things people are doing with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_time': 0.098181818, 'prompt_time': 0.000207219, 'queue_time': 0.01411014, 'total_time': 0.098389037}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fac00f1d-9aa3-479d-819e-1e4c25b1d1ed-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77})

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my my name?")],
    config=config
)

In [16]:
response

AIMessage(content='Your name is Vicky!  I remember you told me at the beginning of our conversation. 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 92, 'total_tokens': 126, 'completion_time': 0.061818182, 'prompt_time': 0.003488628, 'queue_time': 0.010014532999999999, 'total_time': 0.06530681}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9958903-c69c-4693-98b0-46a38e8021e5-0', usage_metadata={'input_tokens': 92, 'output_tokens': 34, 'total_tokens': 126})

In [17]:
## Change the config --> session id
config2 ={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my my name?")],
    config=config2
)
response.content

"As a large language model, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊 \n\n"

### Prompt templates
Prompt Templates helps to turn raw user information into a format that the LLM can work with. In this  case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, Let's add in a system message with some custom instructions(but still taking messages as input). Next, we'll add in more input besides just the messages.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is vicky")]})

AIMessage(content="Hello Vicky!  \n\nIt's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 33, 'total_tokens': 62, 'completion_time': 0.052727273, 'prompt_time': 0.000356599, 'queue_time': 0.013020609999999998, 'total_time': 0.053083872}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9db83d03-353c-48ea-86d2-d8a17db69363-0', usage_metadata={'input_tokens': 33, 'output_tokens': 29, 'total_tokens': 62})

In [21]:
with_message_history = RunnableWithMessageHistory(chain, get_seesion_history)

In [22]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is vicky")],
    config=config
)

In [24]:
response.content

"Hi Vicky! It's nice to meet you.  \n\nWhat can I do for you today? 😊  I'm ready for any questions you have. \n"

In [25]:
## Add more complexity
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the nest of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [27]:
response = chain.invoke({
    "messages":[HumanMessage(content="Hi My name is vicky")],
    "language":"Hindi"
    })
response.content

'नमस्ते विक्की! 😊  मुझे बहुत खुशी है कि आप मेरे साथ बात कर रहे हैं।\n\nक्या मैं आपकी कोई मदद कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a messages History class. THis time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_seesion_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi My name is vicky")],
    "language":"Hindi"
    },
    config=config
)
response.content

'नमस्ते विक्की!  मैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआप क्या जानना चाहते हैं? 😊 \n'

### Managing the conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therfore, it is important to add a step that limits the size of messages you are passing in.


'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.



In [35]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2 ?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun ?"),
    AIMessage(content="yes"),
]

In [40]:
!pip install transformers

In [39]:
trimmer.invoke(messages)

[SystemMessage(content="you're good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import  itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do i like")],
        "language":"English"
    }
)
response.content

'You said you like vanilla ice cream! 🍦  \n\nDo you want to talk about ice cream more?  Maybe your favorite toppings?\n'

In [44]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i ask")],
        "language":"English"
    }
)
response.content

'You asked "what is 2+2?". \n'

In [45]:
# Lets wrap this in the message History

with_message_history = RunnableWithMessageHistory(
    chain,
    get_seesion_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat5"}}

In [46]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="Whats my name?")],
        "language":"English"
    },
    config=config
)
response.content

'You told me your name is Bob!  😊  \n'

### Vector Stores and Retrievers

This Video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrievel of data -- from (vector) databases and other sources -- for integration with LLM workflows. They are important for applications that fatch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.


we will cover
- Documents
- Vector Stores
- Retrievers


In [47]:
!pip install langchain-chroma

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/604.0 kB ?

In [11]:
from langchain_core.documents import Document
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_KEY")

In [7]:
documents = [
    Document(
        page_content="Dogs are great companions, know for their loyality and friendliness.",
        metadata= {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata= {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata= {"source":"fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata= {"source":"bird-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata= {"source":"mammal-pets-doc"}
    ),
]

In [8]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [12]:
os.environ["HF_Token"] = os.getenv("HUGGING_FACE_TOKEN")

In [14]:
## VectorStores
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key= groq_api_key,model= "Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002CB883961B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002CB88396C30>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [55]:
!pip install langchain_huggingface

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/199.4 MB 6.3 MB/s eta 0:00:32
   ---------------------------------------- 1.3/199.4 MB 5.6 MB/s eta 0:00:36
   ---------------------------------------- 2.4/199.4 MB 3.8 MB/s eta 0:00:52
    --------------------------------------- 2.6/199.4 MB 3.9 MB/s eta 0:00:51
    --------------------------------------- 3.7/199.4 MB 3.5 MB/s eta 0:00:57
    --------------------------------------- 4.2/199.4 MB 3.5 MB/s eta 0:00:56
    --------------------------------------- 4.7/199.4 MB 3.2 MB/s eta 0:01:01
   - -------------------------------------- 5.5/199.4 MB 3.2 MB/s eta 0:01:01
   - -------------------------------------- 6.3/199.4 MB 3.2 MB/s eta 0:01:01
   - --------------

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

In [16]:
embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

c:\Users\kumar\anaconda3\envs\ann_project\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:
!pip install sentence-transformers

In [17]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents= documents,embedding=embedding)
vectorstore

In [18]:
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [19]:
#Async Query
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [20]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.'),
  1.5532042980194092),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

### Retrievers
Langcahin vectorStore object do not subclass Runnable, and so cannot immediately be integrated two into Langchain Expression Langchain chains.

Langchain Retrievers are Runnables, so they implement a standard set of method(eg. synchronous and asynchronous invoke and batch operations) and are designed to be incorported in LCEL chain. 

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [21]:
from typing import List

from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.')]]

VectorStores implements an as_retriever method taht will generate a Retriever, specifically a VectorStoreRetriever. Those retrievers include specific search_type and search_kwargs attributes that identify what methods of the underling vector store to call , how to parameterize them. For instance, we can replicate the above with the following:

In [23]:
retriever =vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)

In [25]:
retriever.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyality and friendliness.')]]

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""

In [28]:
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")
response.content

'According to the context, dogs are "great companions, known for their loyalty and friendliness."'